In [1]:
from bs4 import BeautifulSoup
import requests
from time import sleep
import time

import pandas as pd
import json

In [2]:
from selenium import webdriver

In [3]:
# write a function to get all information out at one go
# this function is used for Part 1 webscrap purpose

def jobPostingInfo(soup):
    
    # recruiting_company
    companies_sp = soup.find_all('p',{'name':'company'})
    locations_sp = soup.find_all('p',{'name':'location'})
    employmentTypes_sp = soup.find_all('p',{'name':'employment_type'})
    seniorities_sp = soup.find_all('p',{'name':'seniority'})
    categories_sp = soup.find_all('p',{'name':'category'})
  

    companies = []
    locations = []
    employementTypes = []
    seniorities = []
    categories = []
    
    
    for i in range(0,len(companies_sp)):
        comp = companies_sp[i].text
        companies.append(comp)
        
        try:
            loc = locations_sp[i].text
            locations.append(loc)
        except:
            locations.append('-')
        
        try:
            empType = employmentTypes_sp[i].text
            employementTypes.append(empType)
        except:
            employementTypes.append('-')
            
        try:    
            sen = seniorities_sp[i].text
            seniorities.append(sen)
        except:
            seniorities.append('-')
        
        try:
            cat = categories_sp[i].text
            categories.append(cat)
        except:
            categories.append('-')
            

    # job_title:
    job_titles_sp = soup.find_all('h1',{'name':'job_title'})
    job_titles = []
    for i in range(0, len(job_titles_sp)):
        job = job_titles_sp[i].text
        job_titles.append(job)
        
    # gov_Support:
    govSupport_sp = soup.find_all('section',{'class':'dib tl pl3-l mb3'})
    govSupport = []
    for i in range(0, len(govSupport_sp)):
        gov = govSupport_sp[i].text
        govSupport.append(gov)
        
    # salary_range:
    salaries_sp = soup.find_all('div',{'class':'salary tr pl3'})
    salaries = []
    for i in range(0, len(salaries_sp)):
        sal = salaries_sp[i].text
        salaries.append(sal)  
        
    # job_link:
    joblinks_sp = soup.find_all('a',{'class':'bg-white mb3 w-100 dib v-top pa3 no-underline flex-ns flex-wrap JobCard__card___22xP3'})
    joblinks = []
    for i in range(0, len(joblinks_sp)):
        jblk = joblinks_sp[i].get('href')
        joblinks.append(jblk)
    
    return companies, job_titles, locations, employementTypes, seniorities, categories, govSupport, salaries, joblinks



In [19]:
### Part 1 : to get basic job data info -- ###

# Get driver @ choose whichever driver that you prefer 
driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")
# driver = webdriver.Firefox(executable_path='./geckodriver')

compiled_data = []
for page in range(30,40): # For testing purpose, I scrap 20 pages to see if it's working properly
    ''' 
    We need split webscrape pages among us and have to do this on the same day together for all pages, 
    but individually run different page range as info on this page changes everyday 
    
    ''' 
    
    url = "https://www.mycareersfuture.sg/search?search=data&page={}".format(page)
    
    # Visit relevant page.
    driver.get(url)

    # Wait few second.
    sleep(3)

    # Grab the page source.
    html = driver.page_source
    # print(html)

    soup = BeautifulSoup(html, 'lxml')
    
    compiled_data.append(list(jobPostingInfo(soup)))


driver.close()

In [20]:
### Part 2 : to get job desc + role & responsibility info -- ###


driver = webdriver.Chrome(executable_path="./chromedriver/chromedriver")

addOn_data = []

for i in range(0, len(compiled_data)):
    page_temp = []
    
    for j in range(20): # one page only has 20 records
        
        temp = []
        joblink = compiled_data[i][8][j]
        temp.append(joblink)
        
        joblink_url = "https://www.mycareersfuture.sg"+joblink
        
        
        # Visit relevant page.
        driver.get(joblink_url)

        # Wait few second.
        sleep(3)

        # Grab the page source.
        html = driver.page_source
        # print(html)

        soup2 = BeautifulSoup(html, 'lxml')
        
        postDate = soup2.find('span',{'id':'last_posted_date'}).text
        temp.append(postDate)
        
        closeDate = soup2.find('span',{'id':'expiry_date'}).text
        temp.append(closeDate)

        job_content = soup2.find_all('div',{'id':'content'})
        role_resp = job_content[0].text
        temp.append(role_resp)
        
        try:
            requirements = job_content[1].text
            temp.append(requirements)
        except:
            temp.append('-')
        
        
        page_temp.append(temp)
    
    addOn_data.append(page_temp)

driver.close()


In [21]:
# create dataframe for Part 1 scrapped data ==> compiled_data:

df_main = pd.DataFrame(columns=['Company','JobTitle','Location','EmploymentType','Seniority','Category','GovSupport','SalaryRange','JobLink'])

for i in range(0, len(compiled_data)):
    
    df_jobPosting = pd.DataFrame()
    df_jobPosting['Company'] = compiled_data[i][0]
    df_jobPosting['JobTitle'] = compiled_data[i][1]
    df_jobPosting['Location'] = compiled_data[i][2]
    df_jobPosting['EmploymentType'] = compiled_data[i][3]
    df_jobPosting['Seniority'] = compiled_data[i][4]
    df_jobPosting['Category'] = compiled_data[i][5]
    df_jobPosting['GovSupport'] = compiled_data[i][6]
    df_jobPosting['SalaryRange'] = compiled_data[i][7]
    df_jobPosting['JobLink'] = compiled_data[i][8]

    df_main = pd.concat([df_main, df_jobPosting],axis=0)

In [22]:
# dataframe for Part 1 as below : ( 20 pages x 20 records = 400 records in total )

df_main

,Company,JobTitle,Location,EmploymentType,Seniority,Category,GovSupport,SalaryRange,JobLink
0,ZENITHOPTIMEDIA,"Manager / Senior Manager, Analytics",Central,Permanent ...,Manager,Advertising / Media,Government support available,"$5,000to$10,000Monthly",/job/861e4a60f04d773bc7ad5dd09bfaca0b
1,RMA CONTRACTS PTE. LTD.,Discipline Aide (School / Contract / East),Central,Permanent ...,Manager,Advertising / Media,,"$1,700to$1,800Monthly",/job/71b2f0a12ad68009dcf91c65023f327a
2,DEUTSCHE BANK AKTIENGESELLSCHAFT,Anti Financial Crime – Anti Money Laundering M...,Islandwide,Contract,Non-executive,Education and Training,Government support available,"$4,000to$7,000Monthly",/job/74db565094d103b9e1b7fab5d735d1d6
3,MEMBERSON PTE. LTD.,Application Support Engineer,Islandwide,Contract,Non-executive,Education and Training,Government support available,"$3,000to$6,000Monthly",/job/d9eaa02c47618405b55e7f4de0bafcd2
4,DIAMOND GAS INTERNATIONAL PTE. LTD.,"Manager, Business Development",Central,Permanent ...,Professional,Banking and Finance,,Salary undisclosed,/job/9c82c094ab9fc659c568c3401f037922
5,JABIL CIRCUIT (SINGAPORE) PTE. LTD.,GBC Quote and Variance Analyst,Central,Permanent ...,Professional,Banking and Finance,Government support available,"$3,500to$6,000Monthly",/job/f630f93d95820fe5c00604675bc7732b
6,JABIL CIRCUIT (SINGAPORE) PTE. LTD.,GBC Quote and Variance Analyst,Central,Permanent ...,Executive ...,Information Technology,Government support available,"$3,500to$6,000Monthly",/job/281d778283db8bd4e01c4e1380ff8eaf
7,WEGO PTE. LTD.,Product Analyst,Central,Permanent ...,Executive ...,Information Technology,Government support available,"$2,500to$5,000Monthly",/job/5efaeba4d2e49cc386e5ce1c01016c95
8,Singapore Land Authority,Senior / Principal Geospatial Consultant,East,Permanent ...,Middle Management ...,Marketing / Public Relations ...,,"$2,800to$5,600Monthly",/job/cce97a03b30715bcd60969a2afe3421b
9,STANDARD CHARTERED BANK,"Business Analyst, GCC VAT (1 year contract)",East,Permanent ...,Middle Management ...,Marketing / Public Relations ...,Government support available,"$8,000to$13,000Monthly",/job/8e20ad89e5433669ba8585ce0a4ee6d6


In [23]:
# create dataframe for Part 2 scrapped data ==> addOn_data:

JobLink =[]
PostedDate = []
ClosingDate = []
RoleResponsibility = []
Requirements = []

for i in range(0, len(addOn_data)):
    
    for j in range(0, len(addOn_data[i])): # one page only has 20 records

        JobLink.append(addOn_data[i][j][0])
        PostedDate.append(addOn_data[i][j][1].replace('Posted ',''))
        ClosingDate.append(addOn_data[i][j][2].replace('Closing on ',''))
        RoleResponsibility.append(addOn_data[i][j][3])
        Requirements.append(addOn_data[i][j][4])

df_main_2 = pd.DataFrame()
df_main_2['JobLink'] = JobLink
df_main_2['PostedDate'] = PostedDate
df_main_2['ClosingDate'] = ClosingDate
df_main_2['RoleResponsibility'] = RoleResponsibility
df_main_2['Requirements'] = Requirements

In [24]:
# to verify full text of R&R has been extracted out :

df_main_2['RoleResponsibility'][0]

u'We want an Analytics professional, with an advanced science degree and is proficient in algorithmic analytics, attribution/regression, programming/scripting and the commonly-used programs and software in the data space. You will also be someone who thinks outside of the box, is strategic in your thinking and can work with the business teams to both develop and present the findings. You will also be organised, analytical, articulate and self-motivated individual. You believe in innovating with data and can make it approachable and insight/solution focused. A patient, flexible and team-oriented approach is essential. You will need to work well under pressure and be able to quickly understand problems and be capable of providing innovative, pragmatic and workable solutions. \xa0 Main Duties:  Work with leadership to translate data and analytics strategy to plans and tactics at regional and local market level, and scope out new analytics opportunities with the account teams. Translate id

In [25]:
# dataframe for Part 2 as below :

df_main_2

,JobLink,PostedDate,ClosingDate,RoleResponsibility,Requirements
0,/job/861e4a60f04d773bc7ad5dd09bfaca0b,22 Mar 2018,21 Apr 2018,"We want an Analytics professional, with an adv...",Technical Knowledge: In-depth knowledge of st...
1,/job/71b2f0a12ad68009dcf91c65023f327a,22 Mar 2018,21 Apr 2018,Discipline Aide (School/Contract/East) Minimum...,Requirements: Preferably at least a GCE 'O' L...
2,/job/74db565094d103b9e1b7fab5d735d1d6,22 Mar 2018,21 Apr 2018,"Regulation, Compliance & Anti-Financial Crime ...","Essential Skills Sound technical AML, KYC, in..."
3,/job/d9eaa02c47618405b55e7f4de0bafcd2,21 Mar 2018,20 Apr 2018,· Responsible for the technical delivery...,· Bachelor degree in Computer Science In...
4,/job/9c82c094ab9fc659c568c3401f037922,21 Mar 2018,20 Apr 2018,"Prepare reports of findings, illustrating dat...",Educational qualification - Bachelors/Masters...
5,/job/f630f93d95820fe5c00604675bc7732b,21 Mar 2018,20 Apr 2018,Job Summary Role within the Global Business Ce...,Education & Experience Requirements - Bachelor...
6,/job/281d778283db8bd4e01c4e1380ff8eaf,21 Mar 2018,20 Apr 2018,Job Summary Role within the Global Business Ce...,Education & Experience Requirements - Bachelor...
7,/job/5efaeba4d2e49cc386e5ce1c01016c95,20 Mar 2018,19 Apr 2018,"Wego, the leading travel metasearch site in th...",Requirements: 0-3 years experience in a data...
8,/job/cce97a03b30715bcd60969a2afe3421b,26 Feb 2018,26 Apr 2018,Senior / Principal Geospatial Consultant Job...,-
9,/job/8e20ad89e5433669ba8585ce0a4ee6d6,19 Apr 2018,04 May 2018,We are currently looking for a Business Analys...,Key Requirements • 5-8 years+ experience of s...


In [26]:
# join Part 1 and Part 2 dataframe. Final combined dataframe as below :

df_main_final = pd.merge(df_main, df_main_2, how='left',left_on='JobLink', right_on='JobLink')

In [27]:
df_main_final

,Company,JobTitle,Location,EmploymentType,Seniority,Category,GovSupport,SalaryRange,JobLink,PostedDate,ClosingDate,RoleResponsibility,Requirements
0,ZENITHOPTIMEDIA,"Manager / Senior Manager, Analytics",Central,Permanent ...,Manager,Advertising / Media,Government support available,"$5,000to$10,000Monthly",/job/861e4a60f04d773bc7ad5dd09bfaca0b,22 Mar 2018,21 Apr 2018,"We want an Analytics professional, with an adv...",Technical Knowledge: In-depth knowledge of st...
1,RMA CONTRACTS PTE. LTD.,Discipline Aide (School / Contract / East),Central,Permanent ...,Manager,Advertising / Media,,"$1,700to$1,800Monthly",/job/71b2f0a12ad68009dcf91c65023f327a,22 Mar 2018,21 Apr 2018,Discipline Aide (School/Contract/East) Minimum...,Requirements: Preferably at least a GCE 'O' L...
2,DEUTSCHE BANK AKTIENGESELLSCHAFT,Anti Financial Crime – Anti Money Laundering M...,Islandwide,Contract,Non-executive,Education and Training,Government support available,"$4,000to$7,000Monthly",/job/74db565094d103b9e1b7fab5d735d1d6,22 Mar 2018,21 Apr 2018,"Regulation, Compliance & Anti-Financial Crime ...","Essential Skills Sound technical AML, KYC, in..."
3,MEMBERSON PTE. LTD.,Application Support Engineer,Islandwide,Contract,Non-executive,Education and Training,Government support available,"$3,000to$6,000Monthly",/job/d9eaa02c47618405b55e7f4de0bafcd2,21 Mar 2018,20 Apr 2018,· Responsible for the technical delivery...,· Bachelor degree in Computer Science In...
4,DIAMOND GAS INTERNATIONAL PTE. LTD.,"Manager, Business Development",Central,Permanent ...,Professional,Banking and Finance,,Salary undisclosed,/job/9c82c094ab9fc659c568c3401f037922,21 Mar 2018,20 Apr 2018,"Prepare reports of findings, illustrating dat...",Educational qualification - Bachelors/Masters...
5,JABIL CIRCUIT (SINGAPORE) PTE. LTD.,GBC Quote and Variance Analyst,Central,Permanent ...,Professional,Banking and Finance,Government support available,"$3,500to$6,000Monthly",/job/f630f93d95820fe5c00604675bc7732b,21 Mar 2018,20 Apr 2018,Job Summary Role within the Global Business Ce...,Education & Experience Requirements - Bachelor...
6,JABIL CIRCUIT (SINGAPORE) PTE. LTD.,GBC Quote and Variance Analyst,Central,Permanent ...,Executive ...,Information Technology,Government support available,"$3,500to$6,000Monthly",/job/281d778283db8bd4e01c4e1380ff8eaf,21 Mar 2018,20 Apr 2018,Job Summary Role within the Global Business Ce...,Education & Experience Requirements - Bachelor...
7,WEGO PTE. LTD.,Product Analyst,Central,Permanent ...,Executive ...,Information Technology,Government support available,"$2,500to$5,000Monthly",/job/5efaeba4d2e49cc386e5ce1c01016c95,20 Mar 2018,19 Apr 2018,"Wego, the leading travel metasearch site in th...",Requirements: 0-3 years experience in a data...
8,Singapore Land Authority,Senior / Principal Geospatial Consultant,East,Permanent ...,Middle Management ...,Marketing / Public Relations ...,,"$2,800to$5,600Monthly",/job/cce97a03b30715bcd60969a2afe3421b,26 Feb 2018,26 Apr 2018,Senior / Principal Geospatial Consultant Job...,-
9,STANDARD CHARTERED BANK,"Business Analyst, GCC VAT (1 year contract)",East,Permanent ...,Middle Management ...,Marketing / Public Relations ...,Government support available,"$8,000to$13,000Monthly",/job/8e20ad89e5433669ba8585ce0a4ee6d6,19 Apr 2018,04 May 2018,We are currently looking for a Business Analys...,Key Requirements • 5-8 years+ experience of s...


In [28]:
# verification of scrapped data :

a = df_main_final[df_main_final['JobLink']=='/job/4444308b9cdfe083237649bb58399cda']
a

,Company,JobTitle,Location,EmploymentType,Seniority,Category,GovSupport,SalaryRange,JobLink,PostedDate,ClosingDate,RoleResponsibility,Requirements


In [29]:
print(a['RoleResponsibility'].values)

In [30]:
print(a['Requirements'].values)

In [31]:
# Save scrapped data into CSV 

'''
# import sys is meant for capturing of 'weird' characters when transfering to csv file
# encoding=utf8
# do not put this on the very first top row as other library imports, it will cause print/for-loop error
# in which nothing will be printed or iterating through for-loop

'''

import sys
reload(sys)
sys.setdefaultencoding('utf8')

df_main_final.to_csv('./data_Peien2') # pls name the file according to the range of pages scrapped

In [32]:
## Happy Web-Scrapping ##

In [33]:
df_main_final

,Company,JobTitle,Location,EmploymentType,Seniority,Category,GovSupport,SalaryRange,JobLink,PostedDate,ClosingDate,RoleResponsibility,Requirements
0,ZENITHOPTIMEDIA,"Manager / Senior Manager, Analytics",Central,Permanent ...,Manager,Advertising / Media,Government support available,"$5,000to$10,000Monthly",/job/861e4a60f04d773bc7ad5dd09bfaca0b,22 Mar 2018,21 Apr 2018,"We want an Analytics professional, with an adv...",Technical Knowledge: In-depth knowledge of st...
1,RMA CONTRACTS PTE. LTD.,Discipline Aide (School / Contract / East),Central,Permanent ...,Manager,Advertising / Media,,"$1,700to$1,800Monthly",/job/71b2f0a12ad68009dcf91c65023f327a,22 Mar 2018,21 Apr 2018,Discipline Aide (School/Contract/East) Minimum...,Requirements: Preferably at least a GCE 'O' L...
2,DEUTSCHE BANK AKTIENGESELLSCHAFT,Anti Financial Crime – Anti Money Laundering M...,Islandwide,Contract,Non-executive,Education and Training,Government support available,"$4,000to$7,000Monthly",/job/74db565094d103b9e1b7fab5d735d1d6,22 Mar 2018,21 Apr 2018,"Regulation, Compliance & Anti-Financial Crime ...","Essential Skills Sound technical AML, KYC, in..."
3,MEMBERSON PTE. LTD.,Application Support Engineer,Islandwide,Contract,Non-executive,Education and Training,Government support available,"$3,000to$6,000Monthly",/job/d9eaa02c47618405b55e7f4de0bafcd2,21 Mar 2018,20 Apr 2018,· Responsible for the technical delivery...,· Bachelor degree in Computer Science In...
4,DIAMOND GAS INTERNATIONAL PTE. LTD.,"Manager, Business Development",Central,Permanent ...,Professional,Banking and Finance,,Salary undisclosed,/job/9c82c094ab9fc659c568c3401f037922,21 Mar 2018,20 Apr 2018,"Prepare reports of findings, illustrating dat...",Educational qualification - Bachelors/Masters...
5,JABIL CIRCUIT (SINGAPORE) PTE. LTD.,GBC Quote and Variance Analyst,Central,Permanent ...,Professional,Banking and Finance,Government support available,"$3,500to$6,000Monthly",/job/f630f93d95820fe5c00604675bc7732b,21 Mar 2018,20 Apr 2018,Job Summary Role within the Global Business Ce...,Education & Experience Requirements - Bachelor...
6,JABIL CIRCUIT (SINGAPORE) PTE. LTD.,GBC Quote and Variance Analyst,Central,Permanent ...,Executive ...,Information Technology,Government support available,"$3,500to$6,000Monthly",/job/281d778283db8bd4e01c4e1380ff8eaf,21 Mar 2018,20 Apr 2018,Job Summary Role within the Global Business Ce...,Education & Experience Requirements - Bachelor...
7,WEGO PTE. LTD.,Product Analyst,Central,Permanent ...,Executive ...,Information Technology,Government support available,"$2,500to$5,000Monthly",/job/5efaeba4d2e49cc386e5ce1c01016c95,20 Mar 2018,19 Apr 2018,"Wego, the leading travel metasearch site in th...",Requirements: 0-3 years experience in a data...
8,Singapore Land Authority,Senior / Principal Geospatial Consultant,East,Permanent ...,Middle Management ...,Marketing / Public Relations ...,,"$2,800to$5,600Monthly",/job/cce97a03b30715bcd60969a2afe3421b,26 Feb 2018,26 Apr 2018,Senior / Principal Geospatial Consultant Job...,-
9,STANDARD CHARTERED BANK,"Business Analyst, GCC VAT (1 year contract)",East,Permanent ...,Middle Management ...,Marketing / Public Relations ...,Government support available,"$8,000to$13,000Monthly",/job/8e20ad89e5433669ba8585ce0a4ee6d6,19 Apr 2018,04 May 2018,We are currently looking for a Business Analys...,Key Requirements • 5-8 years+ experience of s...
